In [1]:
import numpy as np
import random
twopi = 2.*np.pi
oneOver2Pi = 1./twopi

In [7]:
#
# Processing parameters
#
run = ""
in_dir = "/home/walterms/project/walterms/mcmd/output/"+run+"/"
out_dir = "/home/walterms/project/walterms/mcmd/nn/data/unlbl/"

# For trnfnames, dict of run: label
# [iso, D, T, X, U, L]
fnames = ["bigbox_vid"]

# nbl parameters
# Use -1 to mean all
nblskip = 1 # Skip first few images
NBL = -1

In [5]:
#### First count blocks of each file
nblList = {}
for f in fnames:
    fin = open(in_dir+f, 'r')
    n = 0
    for line in fin.readlines():
        if line == "\n":
            n+=1
    nblList.update({f: n})
    fin.close()
print nblList

{'bigbox_vid': 4001}


In [6]:
for f in fnames:
    nbl = 0
    print "processing " + f + " for unlabeled data"
    fin = open(in_dir+f, 'r')
    fout = open(out_dir+f, 'w')
    
    # find width from file header
    width, height = 0., 0.
    l = fin.readline().split("|")
    for ll in l:
        if "boxEdge" in ll:
            width = float(ll.split()[1])
            break
    height = width
    fin.seek(0)

    if width == 0.:
        # calculate edge length based on vertices of first block
        block = []
        for line in fin.readlines():
            if line == "\n": break
            if line[0].isalpha(): continue
            block.append(line)
        fin.seek(0)
        width, height = edgeLenCalc(block)

    if not (fin.readline()[0].isalpha()): fin.seek(0)

    normX, normY = 1./width, 1./height # normalize x and y
    thNorm = oneOver2Pi

    fRot = 0.
    # adjust nblunlbl if needed
    nblstop = NBL
    if NBL == -1:
        nblstop = nblList[f]
        
    for line in fin.readlines():
        if nbl < nblskip:
            if line == "\n":
                nbl += 1
            continue
        if line == "\n":
            fout.write("\n")
            nbl+=1
#             fRot = random.randint(0,3)
#             fRot = np.random.rand(1)*4
            if nbl > nblstop:
                break
            else: continue
        spt = [float(x) for x in line.split()]
        x,y,th = spt[2],spt[3],spt[4]
        # Rotate block
        # note thetas should be [0,2pi]
        th_ = fRot*twopi*0.25
        th += th_
        if th > twopi: th-=twopi
        th *= oneOver2Pi

        x = np.cos(th_)*spt[2] - np.sin(th_)*spt[3]
        y = np.sin(th_)*spt[2] + np.cos(th_)*spt[3]
        x *= normX
        y *= normY
        fout.write('%f %f %f\n' % (x, y, th))
    fout.close()
    fin.close()
print "Done"

processing bigbox_vid for unlabeled data
Done


In [ ]:
twopi